In [3]:
import os 
import pandas as pd

## загрузка файлов

def load_data():
    df = pd.read_csv('Full_defolt_bd.csv')
    df['Oтчетная дата'] = pd.to_datetime(df['Oтчетная дата'])
    df['Дата начала'] = pd.to_datetime(df['Дата начала'])
    df['Дефолт'] = df[['Кол-во дней просрочки по ОД',
           'Кол-во дней просрочки по НВ',
           'Кол-во дней просрочки по гарантийному взносу']].max(axis = 1)
    return df
def deploy_data(df_base):
    ask = input('Найдены файлы в папке "Добавить отчет", добавить  (д/н)?\n'+str(dirlist))
    if ask.lower() == 'н':
        print('Удалите файл при отстутсвии необходимости')
        return df_base
    elif ask.lower() == 'д':
        data = []
        print('Читаем excel файл это может занять некоторое время')
        for file in dirlist:
            path = r'./Добавить отчет/'+file
            df = pd.read_excel(path)

            file = pd.ExcelFile(path)
            sheets = file.sheet_names
            new_otchet = pd.to_datetime(sheets[0], format="%d.%m.%y")
            file.close()
            df.columns = df.iloc[2]
            df = df.iloc[3:]
            df = df[df['ID'].notna()]

            df['Oтчетная дата'] = new_otchet
            data.append(df[['ID','Oтчетная дата','Дата начала','Кол-во дней просрочки по ОД','Кол-во дней просрочки по НВ','Кол-во дней просрочки по гарантийному взносу']])
            os.remove(path)
            print(path+"file Removed!")
        df = pd.concat(data)
        df.reset_index(drop = True,inplace = True)
        df['Oтчетная дата'] = pd.to_datetime(df['Oтчетная дата'], format="%d.%m.%y")
        df['Дефолт'] = df[['Кол-во дней просрочки по ОД',
               'Кол-во дней просрочки по НВ',
               'Кол-во дней просрочки по гарантийному взносу']].max(axis = 1)
        for date_otchet in df['Oтчетная дата'].unique():
            if date_otchet in df_base['Oтчетная дата'].to_list():
                ask = input(str(date_otchet)+'\nДанная отчетная дата уже присутствует, заменить (д/н)?')
                if ask.lower() == 'д':
                    df_base = df_base[df_base['Oтчетная дата'] != date_otchet]
                    df_base = pd.concat([df_base,df[df['Oтчетная дата']==date_otchet]])
                    print('Отчет удачно заменен')
                else:
                    continue
        df.to_csv('Full_defolt.csv',index = False)
        print('Отчет добавлен/заменен')
        return df_base
    
def ids(df):
    
    df['Дата начала'] = df['Дата начала'] + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(normalize = True)
    df = df.melt(id_vars = 'Дата начала', value_vars='ID')
    df = df.rename(columns={'Дата начала':'date','value':'id'})[['date','id']]
    df = df.sort_values('date').drop_duplicates('id',keep = 'first')
    df.date = df.date + pd.DateOffset(months=1)
    df.to_csv('ID_new.csv',index=False)
    return df


def vintage(df,ids):
    date_range = pd.date_range(start = df['Oтчетная дата'].max() - pd.DateOffset(years=5), end = df['Oтчетная дата'].max(), freq='MS')
    vintage = []
    for date in date_range:
        id_list = ids[ids['date'] == date]['id'].drop_duplicates().values
        temp = df[(df['ID'].isin(id_list))&(df['Дефолт']>=90)&(df['Дефолт']<=120)].groupby('Oтчетная дата').count()['Дефолт'].reset_index()
        temp.rename(columns ={'Дефолт':date.date()},inplace = True)

        date_range = pd.date_range(start = date,end = df['Oтчетная дата'].max(), freq='MS')
        merge_month = pd.DataFrame({'Oтчетная дата':date_range,'months':[i for i in range(1,len(date_range)+1)]})
        temp = pd.merge(merge_month,temp, on = 'Oтчетная дата',how = 'left')
        temp.drop('Oтчетная дата',axis = 1,inplace = True)
        temp = temp.fillna('-')
        vintage.append(temp)

        print(date)

    df = pd.concat(vintage,axis = 1).drop('months',axis = 1)
    df.index = df.index + 1
    df = df.transpose().loc[:,3:]

    df.index = pd.to_datetime(df.index)


    df = pd.merge(df,ids.date.value_counts().sort_index(),
             left_on = df.index,
            right_on = ids.date.value_counts().sort_index().index)
    df = df.rename(columns = {'key_0':'отчетная дата','date':"Количество"})
    df = df[['отчетная дата','Количество'] + [i for i in range(3,61)]]
    
    date_range = pd.DataFrame({'отчетная дата':date_range})
    df = pd.merge(df,date_range, how = 'outer')
    df['Количество'] = df['Количество'].fillna(0)
    return df





df = load_data()
print('Последняя отчетная дата = ',df['Oтчетная дата'].max())
print('Для загрузки нового отчета поместите файл в папку "Добавить отчет"')
dirlist = os.listdir(path="./Добавить отчет")
if dirlist:
    df = deploy_data(df)
    

ids = ids(df)
df = vintage(df,ids)
df.to_excel('vintage_bd.xlsx')
os.startfile('vintage_bd.xlsx')


Последняя отчетная дата =  2021-11-01 00:00:00
Для загрузки нового отчета поместите файл в папку "Добавить отчет"
Найдены файлы в папке "Добавить отчет", добавить  (д/н)?
['СП для матрицы 01.12.21.xlsx']д
Читаем excel файл это может занять некоторое время
./Добавить отчет/СП для матрицы 01.12.21.xlsxfile Removed!
Отчет добавлен/заменен
2016-11-01 00:00:00
2016-12-01 00:00:00
2017-01-01 00:00:00
2017-02-01 00:00:00
2017-03-01 00:00:00
2017-04-01 00:00:00
2017-05-01 00:00:00
2017-06-01 00:00:00
2017-07-01 00:00:00
2017-08-01 00:00:00
2017-09-01 00:00:00
2017-10-01 00:00:00
2017-11-01 00:00:00
2017-12-01 00:00:00
2018-01-01 00:00:00
2018-02-01 00:00:00
2018-03-01 00:00:00
2018-04-01 00:00:00
2018-05-01 00:00:00
2018-06-01 00:00:00
2018-07-01 00:00:00
2018-08-01 00:00:00
2018-09-01 00:00:00
2018-10-01 00:00:00
2018-11-01 00:00:00
2018-12-01 00:00:00
2019-01-01 00:00:00
2019-02-01 00:00:00
2019-03-01 00:00:00
2019-04-01 00:00:00
2019-05-01 00:00:00
2019-06-01 00:00:00
2019-07-01 00:00:00
20